In [1]:
%load_ext autoreload
%autoreload 2
from geolifeclef.utils import get_spark
from pyspark.sql import functions as F

spark = get_spark()
data_root = "/mnt/data/geolifeclef-2024/data"

survey_stats = spark.read.parquet(
    f"{data_root}/processed/geolsh_nn_graph/v1/survey_stats"
)
species_stats = spark.read.parquet(
    f"{data_root}/processed/geolsh_nn_graph/v1/species_stats"
)
timing = spark.read.parquet(f"{data_root}/processed/geolsh_nn_graph/v1/timing")
survey_stats.printSchema()
species_stats.printSchema()

24/04/06 15:17:02 WARN Utils: Your hostname, daphne-major resolves to a loopback address: 127.0.1.1; using 172.28.199.217 instead (on interface eth0)
24/04/06 15:17:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/06 15:17:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/06 15:17:04 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
24/04/06 15:17:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


root
 |-- summary: string (nullable = true)
 |-- srcSurveyId: string (nullable = true)
 |-- count: string (nullable = true)
 |-- threshold: integer (nullable = true)

root
 |-- summary: string (nullable = true)
 |-- srcSpeciesId: string (nullable = true)
 |-- count: string (nullable = true)
 |-- threshold: integer (nullable = true)



In [9]:
print("survey to species network")
survey_stats.groupBy("threshold").pivot("summary").agg(
    F.first(F.round("count", 2))
).orderBy("threshold").show()

survey to species network


+---------+---------+-----+-----+---+------+
|threshold|    count|  max| mean|min|stddev|
+---------+---------+-----+-----+---+------+
|    10000|3919238.0|273.0|19.84|1.0| 25.98|
|    20000|3919238.0|305.0|27.58|1.0| 33.45|
|    30000|3919238.0|318.0| 33.7|1.0| 39.11|
|    40000|3919238.0|334.0|38.84|1.0| 43.68|
|    50000|3919238.0|351.0|43.33|1.0| 47.29|
|    60000|3919238.0|355.0|47.35|1.0| 50.16|
|    70000|3919238.0|359.0| 51.0|1.0| 52.55|
|    80000|3919238.0|362.0|54.39|1.0| 54.56|
|    90000|3919238.0|367.0|57.61|1.0| 56.39|
|   100000|3919238.0|370.0| 60.6|1.0| 57.94|
+---------+---------+-----+-----+---+------+



In [8]:
print("species to species network")
species_stats.groupBy("threshold").pivot("summary").agg(
    F.first(F.round("count", 2))
).orderBy("threshold").show()

species to species network


+---------+-------+------+-------+---+-------+
|threshold|  count|   max|   mean|min| stddev|
+---------+-------+------+-------+---+-------+
|    10000|10359.0|5083.0| 698.32|1.0| 955.16|
|    20000|10359.0|5345.0| 786.85|1.0|1046.07|
|    30000|10359.0|5540.0| 845.94|1.0|1104.46|
|    40000|10359.0|5682.0| 891.74|1.0|1148.41|
|    50000|10359.0|5810.0| 930.36|1.0|1184.73|
|    60000|10359.0|5914.0| 964.23|1.0|1215.97|
|    70000|10359.0|5980.0| 993.66|1.0|1242.86|
|    80000|10359.0|6043.0|1020.18|1.0| 1267.0|
|    90000|10359.0|6092.0|1044.34|1.0|1288.65|
|   100000|10359.0|6134.0|1065.68|1.0|1307.58|
+---------+-------+------+-------+---+-------+



In [12]:
# check the edges size
survey_edges = spark.read.parquet(
    f"{data_root}/processed/geolsh_nn_graph/v1/survey_edges"
)
species_edges = spark.read.parquet(
    f"{data_root}/processed/geolsh_nn_graph/v1/species_edges"
)
survey_edges.printSchema()
species_edges.printSchema()

print("survey to species edges")
survey_edges.groupBy("threshold").agg(
    F.countDistinct("srcSurveyId", "dstSpeciesId")
).orderBy("threshold").show()

print("species to species edges")
species_edges.groupBy("threshold").agg(
    F.countDistinct("srcSpeciesId", "dstSpeciesId")
).orderBy("threshold").show()

root
 |-- srcSurveyId: integer (nullable = true)
 |-- count: long (nullable = true)
 |-- threshold: integer (nullable = true)

root
 |-- srcSpeciesId: double (nullable = true)
 |-- count: long (nullable = true)
 |-- threshold: integer (nullable = true)

survey to species edges


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `dstSpeciesId` cannot be resolved. Did you mean one of the following? [`srcSurveyId`, `threshold`, `count`].;
'Aggregate [threshold#740], [threshold#740, unresolvedalias('count(distinct srcSurveyId#738, 'dstSpeciesId), None)]
+- Relation [srcSurveyId#738,count#739L,threshold#740] parquet
